Import useful Libraries

In [5]:
pip install pyforest

  Created wheel for pyforest: filename=pyforest-1.0.3-py2.py3-none-any.whl size=13716 sha256=33317f31006ced5b8bb4e55679e63f187e03607a869ae66be81a1cde11063de8
  Stored in directory: /root/.cache/pip/wheels/af/0b/39/340a7f15fc8d4ff5ab50847b28789afea04452a9d51b9721b5
Successfully built pyforest


In [6]:
from transformers import pipeline
import pyforest
import string
from geopy.distance import geodesic
from statistics import mean
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Read data from CSV file through link.

In [7]:
pr_data  = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/product_ratings/train_set_label.csv" )
test_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/product_ratings/test_set_label.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Check for Train Data Information

In [8]:
pr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7486 entries, 0 to 7485
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      7486 non-null   object 
 1   product_name                                 7486 non-null   object 
 2   manufacturer                                 7481 non-null   object 
 3   price                                        6417 non-null   object 
 4   number_available_in_stock                    5587 non-null   object 
 5   number_of_reviews                            7486 non-null   object 
 6   number_of_answered_questions                 6909 non-null   float64
 7   amazon_category_and_sub_category             7000 non-null   object 
 8   customers_who_bought_this_item_also_bought   6665 non-null   object 
 9   description                                  7013 non-null   object 
 10  

Check for Test Data Information

In [9]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2496 entries, 0 to 2495
Data columns (total 16 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      2496 non-null   object 
 1   product_name                                 2496 non-null   object 
 2   manufacturer                                 2494 non-null   object 
 3   price                                        2133 non-null   object 
 4   number_available_in_stock                    1898 non-null   object 
 5   number_of_reviews                            2496 non-null   int64  
 6   number_of_answered_questions                 2309 non-null   float64
 7   amazon_category_and_sub_category             2298 non-null   object 
 8   customers_who_bought_this_item_also_bought   2255 non-null   object 
 9   description                                  2322 non-null   object 
 10  

Check whether Data Types are appropriate or not.

In [10]:
pr_data.dtypes

uniq_id                                         object
product_name                                    object
manufacturer                                    object
price                                           object
number_available_in_stock                       object
number_of_reviews                               object
number_of_answered_questions                   float64
amazon_category_and_sub_category                object
customers_who_bought_this_item_also_bought      object
description                                     object
product_information                             object
product_description                             object
items_customers_buy_after_viewing_this_item     object
customer_questions_and_answers                  object
customer_reviews                                object
sellers                                         object
average_review_rating                          float64
dtype: object

In [11]:
test_data.dtypes

uniq_id                                         object
product_name                                    object
manufacturer                                    object
price                                           object
number_available_in_stock                       object
number_of_reviews                                int64
number_of_answered_questions                   float64
amazon_category_and_sub_category                object
customers_who_bought_this_item_also_bought      object
description                                     object
product_information                             object
product_description                             object
items_customers_buy_after_viewing_this_item     object
customer_questions_and_answers                  object
customer_reviews                                object
sellers                                         object
dtype: object

Process of removing '£' symbol from price colum of the train & test data.

In [12]:
pr_data['price']=pr_data.price.str.replace('£','')
test_data['price']=test_data.price.str.replace('£','')

In number_available_in_stock I found that it contains data in the form of 3 used or 10 new. Here we have to split numeric value and New/Used information.

In [13]:
temp=pr_data.number_available_in_stock.str.split(expand=True,).rename(columns={0:'available',1:'new_used'},errors="raise")
temp_1=test_data.number_available_in_stock.str.split(expand=True,).rename(columns={0:'available',1:'new_used'},errors="raise")

In [14]:
pr_data=pr_data.join(temp)
test_data=test_data.join(temp_1)

Drop irrelevant data columns.

In [15]:
pr_data=pr_data.drop(['number_available_in_stock','product_description','uniq_id'],axis=1)
test_data=test_data.drop(['number_available_in_stock','product_description','uniq_id'],axis=1)

Convert to appropriate data type.

In [16]:
pr_data['price'] = pd.to_numeric(pr_data.price, errors='coerce')
pr_data['number_of_reviews']=pd.to_numeric(pr_data.number_of_reviews,errors='coerce')
pr_data['available']=pd.to_numeric(pr_data.available,errors='coerce')
test_data['price'] = pd.to_numeric(test_data.price, errors='coerce')
test_data['number_of_reviews']=pd.to_numeric(test_data.number_of_reviews,errors='coerce')
test_data['available']=pd.to_numeric(test_data.available,errors='coerce')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Missing Value Analysis

In [17]:
pr_data.isnull().sum()

product_name                                      0
manufacturer                                      5
price                                          1083
number_of_reviews                                 2
number_of_answered_questions                    577
amazon_category_and_sub_category                486
customers_who_bought_this_item_also_bought      821
description                                     473
product_information                              45
items_customers_buy_after_viewing_this_item    2307
customer_questions_and_answers                 6817
customer_reviews                                  3
sellers                                        2314
average_review_rating                             0
available                                      1899
new_used                                       1899
dtype: int64

In [18]:
test_data.isnull().sum()

product_name                                      0
manufacturer                                      2
price                                           368
number_of_reviews                                 0
number_of_answered_questions                    187
amazon_category_and_sub_category                198
customers_who_bought_this_item_also_bought      241
description                                     174
product_information                              13
items_customers_buy_after_viewing_this_item     751
customer_questions_and_answers                 2252
customer_reviews                                  0
sellers                                         766
available                                       598
new_used                                        598
dtype: int64

Missing Value Percentage

In [19]:
missing_value = pd.DataFrame(pr_data.isnull().sum())
missing_value = missing_value.reset_index()
missing_value = missing_value.rename(columns={'index':'variables',0:'percentage_missing_values'})
missing_value['percentage_missing_values'] = missing_value['percentage_missing_values']/len(pr_data)*100

<IPython.core.display.Javascript object>

In [20]:
missing_value

,variables,percentage_missing_values
0,product_name,0.000000
1,manufacturer,0.066791
2,price,14.467005
3,number_of_reviews,0.026717
4,number_of_answered_questions,7.707721
5,amazon_category_and_sub_category,6.492119
6,customers_who_bought_this_item_also_bought,10.967139
7,description,6.318461
8,product_information,0.601122
9,items_customers_buy_after_viewing_this_item,30.817526


In [21]:
missing_value_test = pd.DataFrame(test_data.isnull().sum())
missing_value_test = missing_value_test.reset_index()
missing_value_test = missing_value_test.rename(columns={'index':'variables',0:'percentage_missing_values'})
missing_value_test['percentage_missing_values'] = missing_value_test['percentage_missing_values']/len(test_data)*100

<IPython.core.display.Javascript object>

In [22]:
missing_value_test

,variables,percentage_missing_values
0,product_name,0.000000
1,manufacturer,0.080128
2,price,14.743590
3,number_of_reviews,0.000000
4,number_of_answered_questions,7.491987
5,amazon_category_and_sub_category,7.932692
6,customers_who_bought_this_item_also_bought,9.655449
7,description,6.971154
8,product_information,0.520833
9,items_customers_buy_after_viewing_this_item,30.088141


Drop Colums with more than 30% Missing Values

In [23]:
pr_data=pr_data.drop(['items_customers_buy_after_viewing_this_item','customer_questions_and_answers','sellers'],axis=1)
test_data=test_data.drop(['items_customers_buy_after_viewing_this_item','customer_questions_and_answers','sellers'],axis=1)

Replace Missing Values with appropriate values/assumption.

In [24]:
pr_data['number_of_answered_questions'].fillna(0,inplace=True)
pr_data['amazon_category_and_sub_category'].fillna('Not Applicable',inplace=True)
pr_data['customers_who_bought_this_item_also_bought'].fillna('Not Applicable',inplace=True)
pr_data['description'].fillna('Not Applicable',inplace=True)
pr_data['available'].fillna(0,inplace=True)
pr_data['new_used'].fillna('Not Applicable',inplace=True)

test_data['number_of_answered_questions'].fillna(0,inplace=True)
test_data['amazon_category_and_sub_category'].fillna('Not Applicable',inplace=True)
test_data['customers_who_bought_this_item_also_bought'].fillna('Not Applicable',inplace=True)
test_data['description'].fillna('Not Applicable',inplace=True)
test_data['available'].fillna(0,inplace=True)
test_data['new_used'].fillna('Not Applicable',inplace=True)

In [25]:
pr_data.isnull().sum()

product_name                                     0
manufacturer                                     5
price                                         1083
number_of_reviews                                2
number_of_answered_questions                     0
amazon_category_and_sub_category                 0
customers_who_bought_this_item_also_bought       0
description                                      0
product_information                             45
customer_reviews                                 3
average_review_rating                            0
available                                        0
new_used                                         0
dtype: int64

In [26]:
test_data.isnull().sum()

product_name                                    0
manufacturer                                    2
price                                         368
number_of_reviews                               0
number_of_answered_questions                    0
amazon_category_and_sub_category                0
customers_who_bought_this_item_also_bought      0
description                                     0
product_information                            13
customer_reviews                                0
available                                       0
new_used                                        0
dtype: int64

Drop Missing values as impute of price of product using KNN or other value is not logical at all.

In [27]:
pr_data = pr_data.dropna()
test_data = test_data.dropna()

Reset Index Numbering

In [28]:
pr_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [29]:
temp_df = pr_data.copy()
test_temp_df=test_data.copy()

Function to extract Average rating and review statements from column 'customer_reviews'.

In [30]:
def get_avg_rating(x):
  rating = []
  pro_review = x.split('|')
  for review in pro_review:
    if review.count('//')==4: 
      rating.append(float(review.split("//")[1]))
  try:
    return round(mean(rating),1)
  except:
    return 0

def extract_review(x):
  rating = []
  pro_review = x.split('|')
  for review in pro_review:
    if review.count('//')==4: 
      rating.append(review.split("//")[4].translate(str.maketrans('', '', string.punctuation)))
  try:
    return rating
  except:
    return [""]

In [31]:
temp_df['average_rating_cal'] = temp_df['customer_reviews'].apply(lambda x:get_avg_rating(x))
temp_df['review_single'] = temp_df['customer_reviews'].apply(lambda x:extract_review(x))
test_temp_df['average_rating_cal'] = test_temp_df['customer_reviews'].apply(lambda x:get_avg_rating(x))
test_temp_df['review_single'] = test_temp_df['customer_reviews'].apply(lambda x:extract_review(x))

Drop rows with null reviews.

In [32]:
count = 0
for x in temp_df['review_single']:
  count = count+1
  if len(x) == 0:
    temp_df=temp_df.drop(count-1)

In [33]:
count = 0
for x in test_temp_df['review_single']:
  count = count+1
  if len(x) == 0:
    test_temp_df=test_temp_df.drop(count-1)

In [34]:
temp_df.reset_index(drop=True, inplace=True)

In [35]:
test_temp_df.reset_index(drop=True, inplace=True)

### Sentiment Analysis

In [36]:
model = pipeline('sentiment-analysis')
def sentiment(x):
  score = []
  result = model(x)
  for i in result:
    if i['label']=='NEGATIVE':
      score.append(1-i['score'])
    else:
      score.append(i['score'])
      pass
  return mean(score)

Apply sentement analysis on data

In [37]:
temp_df['sentiment_score'] = temp_df['review_single'].apply(lambda x:sentiment(x))

In [38]:
temp_df

,product_name,manufacturer,price,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,customer_reviews,average_review_rating,available,new_used,average_rating_cal,review_single,sentiment_score
0,Batman - Role Play - Cowl and Batarang,Mattel,14.99,4.0,1.0,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Rubies-Costume-Co-Flas...,"When kids aspire to be like Batman, it's easy ...",Technical Details Item Weight113 g Product Dim...,Item was broken // 3.0 // 2 April 2014 // By\n...,4.5,0.0,Not Applicable,4.5,[ the item itself was delivered on time howeve...,0.500323
1,Melissa & Doug Sticker Collection - Pink,Melissa & Doug,4.76,47.0,1.0,Arts & Crafts > Paper & Stickers,http://www.amazon.co.uk/Melissa-Doug-Sticker-C...,"Product Description Cute couture, adorable ani...",Technical Details Item Weight299 g Product Dim...,Great sticker collection // 5.0 // 19 Jun. 201...,4.7,15.0,new,4.5,[ Lovely sticker collection Each page is theme...,0.638603
2,"Rock and Roll Cards, 30 Team",ClubKing Ltd,4.49,2.0,1.0,Hobbies > Trading Cards & Accessories > Packs ...,http://www.amazon.co.uk/ClubKing-Ltd-Football-...,"1 Pack of 25 Football Cards, 30 rockstars on e...",Technical Details Item Weight100 g Product Dim...,Rock and roll cards // 5.0 // 28 July 2013 // ...,5.0,2.0,new,5.0,[ this were awesome fundraising with a kick so...,0.996933
3,Crayola 16 Pipsqueak Glitter Glues,Crayola,5.99,10.0,1.0,Characters & Brands > Crayola,http://www.amazon.co.uk/Crayola-3-3542-9-Glitt...,Product Description 16 Pipsqueak washable glit...,Technical Details Item Weight186 g Product Dim...,Little fingers big imagination // 5.0 // 4 Mar...,4.2,9.0,new,3.6,[ Bought this for grandson as he wasnt often a...,0.596298
4,MANDALORIAN SUPER COMMANDO - LEGO Star Wars Mi...,Star Wars,11.95,1.0,1.0,Characters & Brands > Star Wars > Toys,http://www.amazon.co.uk/LEGO-Star-Wars-75022-M...,LEGO Star Wars Minifigure. Brand New minifigur...,Technical Details Item Weight9 g Product Dimen...,Four Stars // 4.0 // 25 Feb. 2015 // By\n \...,4.0,0.0,Not Applicable,4.0,[ Yes it did],0.999678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6359,Brookite Triple Nylon Pinwheel,Brookite,9.20,42.0,2.0,Sports Toys & Outdoor > Kites & Flight Toys,http://www.amazon.co.uk/Brookite-HOAI01-Hot-Ai...,Product Description The Triple Nylon Pinwheel ...,Technical Details Item Weight200 g Product Dim...,Pinwheel Magic! // 4.0 // 26 Jun. 2013 // By\n...,4.3,4.0,new,4.4,[ I wanted to brighten up my garden this summe...,0.681673
6360,Oxford Diecast Devon and Somerset F and R MAN ...,Oxford Diecast,16.43,1.0,1.0,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,Not Applicable,"Our model of the very modern fire appliance, r...",Technical Details Item Weight82 g Product Dime...,Oxford 1/76 Devon & Somerset F and R Man Pump ...,4.0,2.0,new,4.0,[ The model is good but is in complete the mod...,0.113502
6361,Polyhedral 7-Die Borealis Dice Set - Sky Blue ...,Chessex,11.01,1.0,1.0,Games > Dice & Dice Games,http://www.amazon.co.uk/Polyhedral | http://ww...,Signature dice are a compilation of the best s...,Technical Details Item Weight54 g Product Dime...,"High quality product, Exceeded expectations //...",5.0,4.0,new,5.0,[ Wonderful quality dice as can be expected fr...,0.993931
6362,Foxmind Map It!,Foxmind,10.00,1.0,1.0,Games > Educational Games,http://www.amazon.co.uk/Wildcard-Games-Mapomin...,Product Description Do you know your Paris fro...,Technical Details Product Dimensions17.1 x 12....,"fun, educational // 4.0 // 14 Jan. 2013 // By\...",4.0,5.0,new,4.0,[ Neatly produced attractive card game very in...,0.871806


In [39]:
temp_df.to_csv("temp_df.csv",index=False)

In [40]:
test_temp_df['sentiment_score'] = test_temp_df['review_single'].apply(lambda x:sentiment(x))

In [41]:
test_temp_df.to_csv("test_temp_df.csv",index=False)

In [42]:
data = temp_df[['price','number_of_reviews','number_of_answered_questions','average_rating_cal','sentiment_score','average_review_rating']]
data_test = test_temp_df[['price','number_of_reviews','number_of_answered_questions','average_rating_cal','sentiment_score']]

In [43]:
X = data.drop('average_review_rating',axis=1).values
y = data['average_review_rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.20, random_state=42)
print(data.shape, X_train.shape, X_test.shape,y_train.shape,y_test.shape)

<IPython.core.display.Javascript object>

(6364, 6) (5091, 5) (1273, 5) (5091,) (1273,)


In [44]:
def get_score(model, X_train, X_test, y_train, y_test): 
        model.fit(X_train, y_train)
        return model.score(X_test,y_test)

def RMSE(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    print(model)
    print('RMSE_test', np.sqrt(mean_squared_error(y_test, y_pred)))
    print('RMSE_train', np.sqrt(mean_squared_error(y_train, y_pred_train)))

In [45]:
print('XGBRegressor',RMSE(XGBRegressor(), X_train, X_test, y_train, y_test))

[19:13:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)


<IPython.core.display.Javascript object>

RMSE_test 0.09964808502540566


<IPython.core.display.Javascript object>

RMSE_train 0.09284244916886601
XGBRegressor None


In [57]:
Xgb = XGBRegressor(subsample= 0.1, reg_alpha= 0.08685113737513521, n_estimators= 500, max_depth= 3, learning_rate=0.05, colsample_bytree= 0.7000000000000001, colsample_bynode=0.7000000000000001, colsample_bylevel=0.9000000000000001)
Xgb.fit(X,y)
pred = Xgb.predict(data_test.values)
pred_results = pd.DataFrame({"prediction" : pred})
pred_results['prediction'] = pred_results['prediction'].apply(lambda x:round(x,1))
pred_results.to_csv("predictions_xgboost.csv",index=False)

[19:22:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


<IPython.core.display.Javascript object>

In [55]:
pred_results['prediction'] = pred_results['prediction'].apply(lambda x:round(x,1))

In [56]:
pred_results

,prediction
0,5.0
1,4.1
2,4.7
3,5.0
4,4.4
...,...
2111,4.0
2112,4.7
2113,5.0
2114,5.0
